# Using the Cost/Likelihood Classes

This example will introduce the cost function methods used for evaluating a simulation. This example will use a cost class (`pybop.SumOfPower`) as an example, but the methods discussed here are transferable to the other cost classes as well as the likelihood classes.

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
import numpy as np
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

First, we need a `Solution` on which to assess the cost. So we will first construct a model, example dataset, parameters and a corresponding `Simulator`. Then we will construct the `Cost` class and show how users can interact with this class.

In [ ]:
model = pybamm.lithium_ion.SPM()
parameter_values = pybamm.ParameterValues("Chen2020")
t_eval = np.linspace(0, 10, 100)
solution = pybamm.Simulation(model, parameter_values=parameter_values).solve(
    t_eval=t_eval
)
dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": solution["Current [A]"](t_eval),
        "Voltage [V]": solution["Voltage [V]"](t_eval),
    }
)

Now that we have the dataset, let's define the parameters for identification.

In [ ]:
parameter_values.update(
    {
        "Negative electrode active material volume fraction": pybop.ParameterInfo(
            initial_value=0.6,
        ),
        "Positive electrode active material volume fraction": pybop.ParameterInfo(
            initial_value=0.6,
        ),
    }
)

Now that we have the model, parameters, and dataset, we can combine them and construct the `Simulator` class. This class forms the basis for evaluating the forward model for the defined fitting process (parameters and operating conditions). We can run the simulator to obtain an example solution.

In [ ]:
simulator = pybop.pybamm.Simulator(
    model,
    parameter_values=parameter_values,
    protocol=dataset,
    output_variables=["Voltage [V]"],
)
inputs = simulator.parameters.to_dict([0.5, 0.5])
solution = simulator.solve(inputs=inputs)

We define a cost with respect to the example dataset. The conventional way to use the cost class is through the `evaluate` method, which is completed below,

In [ ]:
cost = pybop.SumOfPower(dataset)
cost.evaluate(solution, inputs=inputs)

np.float64(0.08964370830440553)

We can obtain the same result using the `Problem` class which first simulates the forward model for the given `inputs`, then it evaluates the cost for resulting solution and inputs. 

In [ ]:
problem = pybop.Problem(simulator, cost)
evaluation = problem.evaluate(inputs)
evaluation.get_values()[0]

np.float64(0.08964370830440553)

The decoupling of the simulator and cost can be helpful in the case where you want to assess the solution across multiple costs (see pybop.WeightedCost for a PyBOP implementation of this), or want to modify the simulator output before computing the cost.

We can create a custom cost function with an additional step (adding some random noise).

In [ ]:
def my_cost(inputs):
    y = simulator.solve(inputs)
    solution = pybop.Solution(inputs)
    solution.set_solution_variable(
        "Voltage [V]",
        data=y["Voltage [V]"].data
        + np.random.normal(0, 0.003, len(y["Voltage [V]"].data)),
    )
    return cost.evaluate(solution, inputs)

In [ ]:
my_cost(inputs)

np.float64(0.08963951979352584)

The above example can be re-implemented with gradient calculations using the `calculate_sensitivities` argument.

In [ ]:
def my_cost_gradient(inputs):
    y = simulator.solve(inputs, calculate_sensitivities=True)
    solution = pybop.Solution(inputs)
    solution.set_solution_variable(
        "Voltage [V]",
        data=y["Voltage [V]"].data
        + np.random.normal(0, 0.003, len(y["Voltage [V]"].data)),
        sensitivities=y["Voltage [V]"].sensitivities,
    )
    return cost.evaluate(solution, inputs, calculate_sensitivities=True)

In [ ]:
my_cost_gradient(inputs)

(np.float64(0.09132023936454056), array([-0.58256263, -0.48798082]))

This provides the cost value for the parameter values, alongside the gradient with respect to each parameter. Finally, the sensitivities can likewise be obtained from the `Problem` class.

In [ ]:
evaluation = problem.evaluate(inputs, calculate_sensitivities=True)
evaluation.get_values()

(array([0.08964371]), array([[-0.5804802 , -0.48654994]]))